Join the data from Part 1 with the data from Part 2 to create a new dataframe.

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:
#Foursqaure API

desired_columns = [ 'name','location', 'distance', 'categories']

Foursquare_df = pd.DataFrame()

for n in range(len(df2)):
    name = df2.at[n, 'name']
    lat = df2.at[n, 'latitude']
    long = df2.at[n, 'longitude']
    free_bikes = df2.at[n, 'free_bikes']
    print("Station Name:", name, "lat:", lat, "long:", long, "Number of free bikes are:", free_bikes)
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius=1000&categories=13064%2C%2013003%2C%2013035%2C%2017114"

    headers = {
        "accept": "application/json",
        "Authorization": "fsq3cuGuLM7++Sr0VNnxkYSDqztfsV3G2yYo4Q1uEwUXYpM= "
    }

    response = requests.get(url, headers=headers)


    data = response.json()
    POI_data = data['results']
    df3 = pd.DataFrame(POI_data)
    df3['location'] = df3['location'].apply(lambda loc: loc['address'])
    df3['categories'] = df3['categories'].apply(lambda x: x[0]['name'] if x else '')
    df3['bike_station_name'] = name
    df3['Latitude'] = lat
    df3['Longitude'] = long
    df3['bike_station_coordinates'] = df3['Latitude'].astype(str) + ', ' + df3['Longitude'].astype(str)
    


    selected_columns = df3[desired_columns]
    display(selected_columns.head())
    Foursquare_df = pd.concat([Foursquare_df,df3])


In [ ]:
#put inplace=true to put changes on the original df
Foursquare_df.reset_index(inplace=True)

In [ ]:
Foursquare_df

In [ ]:
#Yelp API merged with citybikes API

In [ ]:

Yelp_columns = ['name', 'distance', 'location', 'POI_category']

yelp_dataframe = pd.DataFrame()

for n in range(len(df2)):
    name = df2.at[n, 'name']
    lat = df2.at[n, 'latitude']
    long = df2.at[n, 'longitude']
    free_bikes = df2.at[n, 'free_bikes']
    print("Station Name:", name, "lat:", lat, "long:", long, "Number of free bikes are:", free_bikes)
    
    url = f"https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={long}&radius=1000&categories=breakfast_brunch%2Ccoffee%2Cgrocery&sort_by=best_match&limit=20"

    yelp_api = "FSvkEvmyTcQX-tZRriVc9iP4Rjcf0EJwIvhK55IsbwZxCMG1KAkxC-M4xCJGJhywt_56Uz3jOIHr8rz_dbsL5hQoKsJVu0EDDLi7T5xUpMUc1UR4s1Gwb31cmz2gZHYx"
    headers = {
        "accept": "application/json",
        "Authorization": f"bearer {yelp_api}"
    }

    response = requests.get(url, headers=headers)
    data = response.json()

    # Extracting relevant information from the response data
    yelp_POI = data['businesses']
    yelp_df = pd.DataFrame(yelp_POI)
    yelp_df['location'] = yelp_df['location'].apply(lambda loc: loc['address1'])
    yelp_df['bike_station_name'] = name
    yelp_df['Latitude'] = lat
    yelp_df['Longitude'] = long
    yelp_df['POI_category'] = yelp_df['categories'].apply(lambda x: x[0]['title'] if len(x) > 0 else "")
    yelp_df['bike_station_coordinates'] = yelp_df['Latitude'].astype(str) + ', ' + yelp_df['Longitude'].astype(str)
    
    Yelp_selected_columns = yelp_df[Yelp_columns]
    display(Yelp_selected_columns.head())
    yelp_dataframe = pd.concat([yelp_dataframe, yelp_df])
    


In [ ]:
yelp_dataframe.reset_index(inplace=True)

In [ ]:
yelp_dataframe.to_csv(r'C:\Users\nas76\Python-Project\notebooks\yelp1.csv', index=False)

In [ ]:
yelp_dataframe

In [ ]:
'''   
 Compare the quality of the Yelp and Foursquare API. For your location, 
 which API gives you the most complete information/better coverage? 
 NOTE: Your definition of 'coverage' is up to you. It could be simple 'number of POIs in the area', 
 but it could also be something more specific like 'number of reviews per POI', or 'number of different attributes of each POI'.

Yelp gives you more information about the POI since the website allows users to rate the POI. It provides the ratings and the number 
of ratings. This is much more convenient since it can show you the most top rated POI categories we have chosen next to a bike station in toronto''' 

In [ ]:

# Merge the dataframes
merged_dfs = pd.merge(Foursquare_df, yelp_dataframe, on='bike_station_coordinates', how='inner')

# Save merged dataframe to CSV
merged_dfs.to_csv(r'C:\Users\nas76\Python-Project\merged_data_updated.csv', index=False)



# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('yelp_poi_database.db')
yelp_table_schema = """
CREATE TABLE IF NOT EXISTS yelp_pois (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    distance REAL,
    location TEXT,
    poi_category TEXT,
    bike_station_name TEXT,
    latitude REAL,
    longitude REAL,
    bike_station_coordinates TEXT
);
"""

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('foursqaure_poi_database.db')

foursquare_table_schema = """
CREATE TABLE IF NOT EXISTS foursquare_pois (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    location TEXT,
    distance REAL,
    categories TEXT,
    bike_station_name TEXT,
    latitude REAL,
    longitude REAL,
    bike_station_coordinates TEXT
);
"""

Look at the data before and after the join to validate your data.